# Photometric error stage demo

author: Tianqing Zhang, John-Franklin Crenshaw

This notebook demonstrate the use of `rail.creation.degraders.photometric_errors`, which adds column for the photometric noise to the catalog based on the package PhotErr developed by John-Franklin Crenshaw. The RAIL stage PhotoErrorModel inherit from the Noisifier base classes, and the LSST, Roman, Euclid child classes inherit from the PhotoErrorModel.

If you're interested in running this in pipeline mode, see [`02_Photometric_Realization_with_Other_Surveys.ipynb`](https://github.com/LSSTDESC/rail/blob/main/pipeline_examples/creation_examples/02_Photometric_Realization_with_Other_Surveys.ipynb) in the `pipeline_examples/core_examples/` folder.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from rail.core.data import PqHandle
from rail.interactive.creation.degraders import photometric_errors

### Create a random catalog with ugrizy+YJHF bands as the the true input

In [ ]:
data = np.random.normal(23, 3, size=(1000, 9))

data_df = pd.DataFrame(
    data=data, columns=["u", "g", "r", "i", "z", "y", "Y", "J", "H"]  # values
)
data_truth = PqHandle("input")
data_truth.set_data(data_df)

In [ ]:
data_df

### The LSST error model adds noise to the optical bands

In [ ]:
samples_w_errs = photometric_errors.lsst_error_model(sample=data_truth)

samples_w_errs["output"]

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), dpi=100)

for band in "ugrizy":
    # pull out the magnitudes and errors
    mags = samples_w_errs["output"][band].to_numpy()
    errs = samples_w_errs["output"][band + "_err"].to_numpy()

    # sort them by magnitude
    mags, errs = mags[mags.argsort()], errs[mags.argsort()]

    # plot errs vs mags
    ax.plot(mags, errs, label=band)

ax.legend()
ax.set(xlabel="Magnitude (AB)", ylabel="Error (mags)")
plt.show()

### The Roman error model adds noise to the infrared bands

In [ ]:
samples_w_errs_roman = photometric_errors.roman_error_model(
    sample=data_truth,
    m5={"Y": 27.0, "J": 26.4, "H": 26.4},
    theta={"Y": 27.0, "J": 0.106, "H": 0.128},
)

samples_w_errs_roman["output"]

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), dpi=100)

for band in "YJH":
    # pull out the magnitudes and errors
    mags = samples_w_errs_roman["output"][band].to_numpy()
    errs = samples_w_errs_roman["output"][band + "_err"].to_numpy()

    # sort them by magnitude
    mags, errs = mags[mags.argsort()], errs[mags.argsort()]

    # plot errs vs mags
    ax.plot(mags, errs, label=band)

ax.legend()
ax.set(xlabel="Magnitude (AB)", ylabel="Error (mags)")
plt.show()

### The Euclid error model adds noise to YJH bands

In [ ]:
samples_w_errs_Euclid = photometric_errors.euclid_error_model(sample=data_truth)

samples_w_errs_Euclid["output"]

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4), dpi=100)

for band in "YJH":
    # pull out the magnitudes and errors
    mags = samples_w_errs_Euclid["output"][band].to_numpy()
    errs = samples_w_errs_Euclid["output"][band + "_err"].to_numpy()

    # sort them by magnitude
    mags, errs = mags[mags.argsort()], errs[mags.argsort()]

    # plot errs vs mags
    ax.plot(mags, errs, label=band)

ax.legend()
ax.set(xlabel="Magnitude (AB)", ylabel="Error (mags)")
plt.show()